In [1]:
pip install dash


  Obtaining dependency information for dash from https://files.pythonhosted.org/packages/72/ef/d46131f4817f18b329e4fb7c53ba1d31774239d91266a74bccdc932708cc/dash-2.18.2-py3-none-any.whl.metadata
  Obtaining dependency information for dash-html-components==2.0.0 from https://files.pythonhosted.org/packages/75/65/1b16b853844ef59b2742a7de74a598f376ac0ab581f0dcc34db294e5c90e/dash_html_components-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for dash-core-components==2.0.0 from https://files.pythonhosted.org/packages/00/9e/a29f726e84e531a36d56cff187e61d8c96d2cc253c5bcef9a7695acb7e6a/dash_core_components-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for dash-table==5.0.0 from https://files.pythonhosted.org/packages/da/ce/43f77dc8e7bbad02a9f88d07bf794eaf68359df756a28bb9f2f78e255bb1/dash_table-5.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for retrying from https://files.pythonhosted.org/packages/8f/04/9e36f28be4c0532c0e9207ff9dc01fb

In [5]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Load and preprocess the data
accident_data = pd.read_csv("accidents.csv")
weather_data = pd.read_csv("dubai 2023-03-09 to 2024-09-25-2.csv")

# Convert times to datetime
accident_data['acci_time'] = pd.to_datetime(accident_data['acci_time'], format='%d/%m/%Y %H:%M:%S')
weather_data['datetime'] = pd.to_datetime(weather_data['datetime'])

# Merge datasets
merged_data = pd.merge_asof(
    accident_data.sort_values('acci_time'),
    weather_data.sort_values('datetime'),
    left_on='acci_time',
    right_on='datetime',
    direction='backward'
)

# Add new features
merged_data['hour'] = merged_data['acci_time'].dt.hour
merged_data['date'] = merged_data['acci_time'].dt.date
merged_data['temp_celsius'] = (merged_data['temp'] - 32) * 5.0 / 9.0

# Initialize Dash app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("Interactive Accident and Weather Dashboard"),
    
    # Dropdown to select accident category
    html.Label("Select Accident Category:"),
    dcc.Dropdown(
        id='category-dropdown',
        options=[{'label': cat, 'value': cat} for cat in merged_data['acci_name'].unique()],
        value=None,
        placeholder="Select a category"
    ),
    
    # Slider to filter temperature
    html.Label("Filter by Temperature (°C):"),
    dcc.RangeSlider(
        id='temp-slider',
        min=int(merged_data['temp_celsius'].min()),
        max=int(merged_data['temp_celsius'].max()),
        step=1,
        marks={i: f"{i}°C" for i in range(int(merged_data['temp_celsius'].min()), int(merged_data['temp_celsius'].max()) + 1, 5)},
        value=[int(merged_data['temp_celsius'].min()), int(merged_data['temp_celsius'].max())]
    ),
    
    # Heatmap
    dcc.Graph(id='heatmap'),
    
    # Scatter map
    dcc.Graph(id='scatter-map'),
    
    # Timeline
    dcc.Graph(id='timeline')
])

# Callbacks for interactivity
@app.callback(
    [Output('heatmap', 'figure'),
     Output('scatter-map', 'figure'),
     Output('timeline', 'figure')],
    [Input('category-dropdown', 'value'),
     Input('temp-slider', 'value')]
)
def update_graphs(selected_category, temp_range):
    # Filter data
    filtered_data = merged_data[
        (merged_data['temp_celsius'] >= temp_range[0]) &
        (merged_data['temp_celsius'] <= temp_range[1])
    ]
    if selected_category:
        filtered_data = filtered_data[filtered_data['acci_name'] == selected_category]
    
    # Heatmap
    heatmap_data = filtered_data.groupby(['hour', 'temp_celsius']).size().reset_index(name='count')
    heatmap_fig = px.density_heatmap(
        heatmap_data,
        x="temp_celsius",
        y="hour",
        z="count",
        color_continuous_scale="YlGnBu",
        labels={"temp_celsius": "Temperature (°C)", "hour": "Hour of the Day", "count": "Accident Count"},
        title="Accident Frequency by Hour and Temperature"
    )
    
    # Scatter map
    scatter_map_fig = px.scatter_mapbox(
        filtered_data,
        lat="acci_x",
        lon="acci_y",
        color="conditions",
        size="humidity",
        hover_data=["acci_name", "datetime", "humidity", "temp_celsius"],
        title="Accidents and Weather Conditions",
        mapbox_style="carto-positron",
        zoom=10
    )
    
    # Timeline
    timeline_data = filtered_data.groupby(filtered_data['acci_time'].dt.date).agg(
        accident_count=('acci_id', 'count'),
        avg_precip=('precip', 'mean')
    ).reset_index()
    timeline_fig = px.line(
        timeline_data,
        x="acci_time",
        y=["accident_count", "avg_precip"],
        labels={"value": "Count / Precipitation", "variable": "Metric"},
        title="Accident Counts and Average Precipitation Over Time",
        color_discrete_sequence=["blue", "orange"]
    )
    
    return heatmap_fig, scatter_map_fig, timeline_fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


1. Heatmap: Accident Frequency by Hour and Temperature

Description:

X-Axis: Represents the temperature in degrees Celsius (°C).
Y-Axis: Represents the hour of the day (0-23).
Color Intensity: Indicates the count of accidents (darker colors represent higher accident frequencies).

Purpose:

This graph helps visualize how the frequency of accidents changes based on the time of day and temperature. For example:
Accidents may peak during rush hours or extreme temperatures.
Cooler or hotter times of the day might correlate with increased accident counts.

Interactivity:

Users can filter the data using the temperature slider and/or accident category dropdown.
This allows for a focused analysis, such as identifying when "بسيط (Simple)" accidents occur more often.

2. Scatter Map: Accidents and Weather Conditions

Description:

Markers (Points): Represent accident locations on a map.
Marker Size: Proportional to humidity levels.
Marker Color: Encodes the weather conditions (e.g., Clear, Rainy).
Hover Information: Displays detailed accident information:

Accident type
Date and time
Humidity and temperature

Purpose:

This graph provides a spatial view of accidents, showing where they occur relative to weather conditions.
It's useful for:
Identifying accident hotspots.
Correlating accidents with specific weather patterns (e.g., high humidity or rain).

Interactivity:

Users can filter the map based on accident category and temperature range.
For example, selecting "Rainy" conditions highlights locations where accidents happened under wet weather.

3. Timeline: Accident Counts and Average Precipitation Over Time

Description:

X-Axis: Represents dates.
Y-Axis: Dual axis:
Accident count (blue line).
Average precipitation (orange line).

Purpose:

Tracks how accident counts and precipitation levels change over time.

It helps identify:

Long-term trends, such as seasonal peaks in accidents.
Correlations between precipitation levels and accident occurrences (e.g., higher accidents on rainy days).

Interactivity:

The timeline is updated dynamically based on user-selected filters (e.g., category and temperature).
This allows users to observe trends for specific accident types or weather conditions.
